In [41]:
import aocd
import re
from dotenv import load_dotenv
import numpy as np
from tqdm import tqdm
import networkx as nx

load_dotenv()

puzzle = aocd.get_puzzle(day=15, year=2024)
text = puzzle.input_data
examples = puzzle.examples
print(examples)

lines = text.splitlines()

example = examples[0].input_data.splitlines()

example


[Example(input_data='##########\n#..O..O.O#\n#......O.#\n#.OO..O.O#\n#..O@..O.#\n#O#..O...#\n#O..O..O.#\n#.OO.O.OO#\n#....O...#\n##########\n\n<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^\nvvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v\n><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<\n<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^\n^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><\n^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^\n>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^\n<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>\n^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>\nv^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^', answer_a='2028', answer_b=None, extra=None)]


['##########',
 '#..O..O.O#',
 '#......O.#',
 '#.OO..O.O#',
 '#..O@..O.#',
 '#O#..O...#',
 '#O..O..O.#',
 '#.OO.O.OO#',
 '#....O...#',
 '##########',
 '',
 '<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^',
 'vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v',
 '><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<',
 '<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^',
 '^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><',
 '^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^',
 '>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^',
 '<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>',
 '^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>',
 'v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^']

In [39]:
lines = example
text = examples[0].input_data

small = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

small = """#######
#...#.#
#.....#
#..OO.#
#..O..#
#..@..#
#######

^^^"""


In [43]:
m, inst = text.split("\n\n")

m = m.replace("#", "##").replace("O", "[]").replace(".", "..").replace("@", "@.")


m = m.splitlines()
inst = list(inst.replace("\n", ""))

arr = np.array([list(x) for x in m])


move2dir = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}

def move_h(arr, p, new_p, dir):
    p1 = new_p.copy()

    stack = []
    while arr[*new_p] in "[]":
        stack.append(arr[*new_p])
        new_p = new_p + dir

    if arr[*new_p] == "#":
        return

    if arr[*new_p] == ".":
        arr[*p] = "."
        arr[*p1] = "@"        
        for i, s in enumerate(stack, 1):
            arr[*p1 + i*np.array(dir)] = s
    
def move_v(arr, p, new_p, dir):
    if arr[*new_p] == "[":
        to_visit = [tuple(new_p), tuple(new_p + (0, 1))]
    else:
        to_visit = [tuple(new_p), tuple(new_p + (0, -1))]
    
    to_move = [(tuple(p), arr[*p])]
    while to_visit:   
        cp = np.array(to_visit.pop())

        m = (tuple(cp), arr[*cp])

        if m not in to_move:
            to_move.append(m)

        next_p = cp + dir
        if arr[*next_p] == "#":
            return
        
        if arr[*next_p] in "[]":
            to_visit.append(tuple(next_p))
            if arr[*next_p] == "[":
                cpp = next_p + (0, 1)
                if tuple(cpp) not in to_visit:
                    to_visit.append(tuple((next_p + (0, 1))))
            else:
                cpp = next_p + (0, -1)
                if tuple(cpp) not in to_visit:
                    to_visit.append(tuple(next_p + (0, -1)))


    for p, _ in to_move:
        arr[*p] = "."
    
    for p, c in to_move:
        arr[*(np.array(p) + dir)] = c

        
        





def move(arr, move):
    dir = move2dir[move]

    p = np.where(arr == "@")
    p = np.array((p[0][0], p[1][0]))
    new_p = p + dir

    if arr[*new_p] == "#":
        return

    if arr[*new_p] == ".":
        arr[*p] = "."
        arr[*new_p] = "@"
        return

    
    if arr[*new_p] in "[]":
        if move in "<>":
            move_h(arr, p, new_p, dir)
        else:
            move_v(arr, p, new_p, dir)

        
                
                

            


for i in tqdm(inst):
    move(arr, i)



def gps(arr):
    x, y = np.where(arr == "[")
    return x.sum()*100 +  y.sum()

print(gps(arr))

  0%|          | 0/20000 [00:00<?, ?it/s]

100%|██████████| 20000/20000 [00:00<00:00, 37042.88it/s]

1561175


In [ ]:
aocd.submit(gps(arr), day=15, year=2024)

answer a: 1552879
submitting for part b (part a is already completed)
coerced int64 value np.int64(1561175) for 2024/15 to '1561175'


That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 15! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


Gtk-Message: 20:54:00.405: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.


In [ ]:
m, inst = text.split("\n\n")

m = m.splitlines()
inst = list(inst.replace("\n", ""))

arr = np.array([list(x) for x in m])

x, y = np.where(arr == "@")

move2dir = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}


def move(arr, move):
    dir = move2dir[move]

    p = np.where(arr == "@")
    p = np.array((p[0][0], p[1][0]))
    new_p = p + dir

    if arr[*new_p] == "#":
        return

    if arr[*new_p] == ".":
        arr[*p] = "."
        arr[*new_p] = "@"
        return

    if arr[*new_p] == "O":
        p1 = new_p.copy()
        while arr[*new_p] == "O":
            new_p = new_p + dir

        if arr[*new_p] == "#":
            return

        if arr[*new_p] == ".":
            arr[*p] = "."
            arr[*p1] = "@"
            arr[*new_p] = "O"
            return


for i in inst:
    move(arr, i)


for i in arr:
    print("".join(i))


def gps(arr):
    x, y = np.where(arr == "O")
    return x.sum()*100 +  y.sum()

print(gps(arr))